In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections;
using System.Threading;

In [ ]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [ ]:
var input = await File.ReadAllLinesAsync("input.txt");

In [ ]:
public interface State
{
    bool IsGoal {get;}
    string Id {get;}
    IEnumerable<(State state, long cost)> Neighbours();
}

In [ ]:
public class Dijkstra
{
    public Dictionary<string, long> Cost = new Dictionary<string, long>();
    public PriorityQueue<State, long> Queue = new PriorityQueue<State, long>();

    public int Count = 0;
    
    public long Run(State start)
    {
        Queue.Enqueue(start, 0);

        State node;
        long cost;
        while(Queue.TryDequeue(out node, out cost))
        {            
            Count++;

            if(Count % 100000 == 0)
            {
                Debug.Print($"Visited {Count} nodes. {Queue.Count} left");
            }
            
            if(node.IsGoal)
            {
                break;
            }
            
            foreach(var n in node.Neighbours())
            {
                long total = n.cost + cost;

                if(Cost.TryGetValue(n.state.Id, out var c))
                {
                    if(c <= total)
                    {
                        continue;
                    }
                }
                Cost[n.state.Id] = total;
                Queue.Enqueue(n.state, total);
            }
        }
        Debug.Print("Done?");

        if(node?.IsGoal == true)
        {
            Debug.Print("YES!");
            return cost;
        }
        else{
            Debug.Print("NO!");
            return -1;
        }
    }
}

In [ ]:
public class Node
{
    public string Name {get; set;}
    public List<(string destination, int cost)> Destinations {get; set;} = new List<(string, int)>();    
}

In [ ]:
public static class Graph
{
    public static Dictionary<string, Node> Nodes {get; set;} = new Dictionary<string, Node>();

    public static void Load(IEnumerable<string> input)
    {
        foreach(var line in input)
        {
            var segments = line.Split(new [] {"to", "="}, 3, StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries );

            if(!Nodes.ContainsKey(segments[0]))
            {
                Nodes[segments[0]] = new Node{Name = segments[0]};
            }

            if(!Nodes.ContainsKey(segments[1]))
            {
                Nodes[segments[1]] = new Node{Name = segments[1]};
            }

            Nodes[segments[1]].Destinations.Add((segments[0], int.Parse(segments[2])));
            Nodes[segments[0]].Destinations.Add((segments[1], int.Parse(segments[2])));
        }
    }
}

In [ ]:
public class Path : State
{
    public List<String> Nodes {get; set;}
    public bool IsGoal => false;
    public string Id => (Nodes.Count == Graph.Nodes.Count ? "+" : "") + string.Join(',', Nodes);

    public IEnumerable<(State state, long cost)> Neighbours()
    {
        if(!Nodes.Any())
        {
            foreach(var node in Graph.Nodes.Values)
            {
                var path = new Path
                {
                    Nodes = new List<string>(){ node.Name }
                };

                yield return (path, 0);
            }

            yield break;
        }

        foreach(var (destination, cost) in Graph.Nodes[Nodes.Last()].Destinations)
        {
            if(!Nodes.Contains(destination))
            {
                var path = new Path
                {
                    Nodes = Nodes.Concat(new [] {destination}).ToList()
                };

                yield return (path, cost);
            }
        }
    }
}

In [ ]:
Graph.Load(input);

In [ ]:
var start = new Path
{
    Nodes = new List<string>()
};

var d = new Dijkstra();
using(new Debug())
{
    d.Run(start).Display();
}

Visited 100000 nodes. 7683 left
Done?
NO!


-1

In [ ]:
d.Cost.Where(c => c.Key.Contains('+')).Max(c => c.Value)

909